## 1. Run retrosynthesis planning

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
import pandas as pd

from CGRtools import smiles
from IPython.display import SVG, display

from Synto.mcts.tree import Tree, TreeConfig
from Synto.mcts.expansion import PolicyConfig, PolicyFunction

from Synto.interfaces.visualisation import path_graph

In [2]:
policy_config = PolicyConfig(weights_path='/data/tagir/main/syntool/ranking_policy/weights/policy_network.ckpt')
policy_function = PolicyFunction(policy_config=policy_config, compile=False)

In [3]:
tree_config = TreeConfig(
    search_strategy="expansion_first",
    evaluation_mode="rollout",
    min_mol_size=0,
    ucb_type="puct",
    c_ucb=0.1
)

In [4]:
aizy_stats = pd.read_csv("aizy_results_3.5_4.5_test_100.csv")

In [5]:
mols = aizy_stats.target.to_list()

In [11]:
target = smiles(mols[0])
target.canonicalize()
target.clean2d()

In [12]:
target

In [13]:
tree = Tree(
    target=target, 
    reaction_rules_path='/data/tagir/main/syntool/rules/thesis_rules_filtered.pickle',
    building_blocks_path='/data/tagir/main/syntool/tree_data/askcos_bbs.pickle',
    tree_config=tree_config,
    policy_function=policy_function,
)

tree_solved = False
for solved, node_id in tree:
    if solved:
        tree_solved = True

  0%|          | 0/100 [00:00<?, ?it/s]

In [14]:
tree

Tree for: c12n(c(NNC=C3C(N(C(=S)NC3=O)C(C)CC)=O)nc1N(C(N(C)C2=O)=O)C)C
Number of nodes: 777
Number of visited nodes: 46
Found paths: 3
Time: 9.2 seconds

## 2. Analyze planning results

In [10]:
tree = found[0]

In [11]:
print(tree)

Tree for: O[As](O)(c1ccc(cc1)NN=C2C(N=C(N=C2N)SC)=O)=O
Number of nodes: 1626
Number of visited nodes: 100
Found paths: 148
Time: 42.0 seconds


### We can get a list of winning nodes (IDs of leaf nodes in found retrosynthetic paths) by:


## 3. Retrosynthesis path visualisation

In [15]:
for n, node_id in enumerate(tree.winning_nodes):
    print(f'Found path')
    display(SVG(path_graph(tree, node_id)))
    if n == 5:
        break

Found path


Found path


Found path


In [16]:
tree.winning_nodes

[201, 264, 201]

In [17]:
success_node = tree.winning_nodes[0]

tree.path_to_node(success_node)

[retrons_to_expand: (c12n(c(NNC=C3C(N(C(=S)NC3=O)C(C)CC)=O)nc1N(C(N(C)C2=O)=O)C)C,)
 new_retrons: (c12n(c(NNC=C3C(N(C(=S)NC3=O)C(C)CC)=O)nc1N(C(N(C)C2=O)=O)C)C,),
 retrons_to_expand: (c12n(c(NNC=C(C(NC(NC(CC)C)=S)=O)C(=O)O)nc1N(C(N(C)C2=O)=O)C)C,)
 new_retrons: (c12n(c(NNC=C(C(NC(NC(CC)C)=S)=O)C(=O)O)nc1N(C(N(C)C2=O)=O)C)C,),
 retrons_to_expand: (c12n(c(NNC=C(C(=O)N)C(=O)O)nc1N(C(N(C)C2=O)=O)C)C,)
 new_retrons: (S=C=NC(C)CC, c12n(c(NNC=C(C(=O)N)C(=O)O)nc1N(C(N(C)C2=O)=O)C)C),
 retrons_to_expand: (c12n(c(NNC=C(C(=O)N)C(=O)OCC)nc1N(C(N(C)C2=O)=O)C)C,)
 new_retrons: (c12n(c(NNC=C(C(=O)N)C(=O)OCC)nc1N(C(N(C)C2=O)=O)C)C,),
 retrons_to_expand: (CN(C)C=C(C(=O)N)C(OCC)=O, c12n(C)c(nc1N(C(N(C)C2=O)=O)C)NN)
 new_retrons: (CN(C)C=C(C(=O)N)C(OCC)=O, c12n(C)c(nc1N(C(N(C)C2=O)=O)C)NN),
 retrons_to_expand: (c12n(C)c(nc1N(C(N(C)C2=O)=O)C)NN,)
 new_retrons: (C(=O)(CC(=O)N)OCC, COC(OC)N(C)C),
 retrons_to_expand: ()
 new_retrons: (NN, Brc1n(c2C(=O)N(C(N(C)c2n1)=O)C)C)]

In [18]:
tree.synthesis_path(success_node)

(<Synto.chem.reaction.Reaction at 0x7f0c0085baf0>,
 <Synto.chem.reaction.Reaction at 0x7f0c0085bd20>)

In [19]:
tree.path_score(success_node)

0.0

# Run search with all 100 molecules

In [21]:
found = []
for n, mol in enumerate(mols):
    target = smiles(mol)
    target.canonicalize()
    target.clean2d()

    tree = Tree(
        target=target, 
        reaction_rules_path='/data/tagir/main/syntool/rules/rules_extended_filtered.pickle',
        building_blocks_path='/data/tagir/main/syntool/tree_data/askcos_bbs.pickle',
        tree_config=tree_config,
        policy_function=policy_function,
    )
    tree_solved = False
    for solved, node_id in tree:
        if solved:
            tree_solved = True
    
    if tree_solved:
        found.append(tree)
        print(f"Target_{n}_is_solved")
    else:
        del tree 

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 